In [1]:
import pyciemss
import torch

In [3]:
### SETUP ###

model_1_path = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/SEIRHD_NPI_Type1_petrinet.json"
start_time = 0.0
end_time = 100.
logging_step_size = 10.0
num_samples = 3

In [3]:
### SAMPLE INTERFACE ###

result = pyciemss.sample(model_1_path, end_time, logging_step_size, num_samples, start_time=start_time)
result["unprocessed_result"]

{'persistent_beta_c': tensor([0.1562, 0.6827, 0.6109]),
 'persistent_kappa': tensor([0.2488, 0.1847, 0.2561]),
 'persistent_gamma': tensor([0.1372, 0.4412, 0.1647]),
 'persistent_hosp': tensor([0.1155, 0.1856, 0.0822]),
 'persistent_death_hosp': tensor([0.0893, 0.0729, 0.0328]),
 'persistent_I0': tensor([10.9307,  7.7382, 11.9093]),
 'D_state': tensor([[2.3337e-01, 9.3766e-01, 2.2930e+00, 4.8202e+00, 9.5215e+00, 1.8265e+01,
          3.4526e+01, 6.4761e+01, 1.2095e+02],
         [4.1434e-01, 8.3219e-01, 1.0124e+00, 1.0778e+00, 1.1003e+00, 1.1078e+00,
          1.1103e+00, 1.1112e+00, 1.1114e+00],
         [7.0262e-02, 2.6103e-01, 5.8549e-01, 1.1180e+00, 1.9892e+00, 3.4144e+00,
          5.7455e+00, 9.5581e+00, 1.5793e+01]]),
 'E_state': tensor([[3.9104e+01, 7.2340e+01, 1.3455e+02, 2.5025e+02, 4.6542e+02, 8.6551e+02,
          1.6092e+03, 2.9907e+03, 4.3152e+03],
         [1.0361e+01, 3.4270e+00, 1.1363e+00, 3.7680e-01, 1.2494e-01, 4.1431e-02,
          1.3738e-02, 4.5555e-03, 1.4481e-0

In [5]:
result['data'].head()

,timepoint_id,sample_id,persistent_beta_c_param,persistent_kappa_param,persistent_gamma_param,persistent_hosp_param,persistent_death_hosp_param,persistent_I0_param,D_state_state,E_state_state,H_state_state,I_state_state,R_state_state,S_state_state,infected_observable_state,exposed_observable_state,hospitalized_observable_state,dead_observable_state
0,0,0,0.156154,0.248788,0.13725,0.115549,0.089293,10.930715,0.233373,39.103931,2.616503,48.610104,42.412811,19339908.0,48.610104,39.103931,2.616503,0.233373
1,1,0,0.156154,0.248788,0.13725,0.115549,0.089293,10.930715,0.937660,72.339561,5.444248,90.732910,131.613022,19339742.0,90.732910,72.339561,5.444248,0.937660
2,2,0,0.156154,0.248788,0.13725,0.115549,0.089293,10.930715,2.293007,134.547119,10.207091,168.768234,298.074860,19339428.0,168.768234,134.547119,10.207091,2.293007
3,3,0,0.156154,0.248788,0.13725,0.115549,0.089293,10.930715,4.820197,250.249542,18.996023,313.904022,607.757568,19338846.0,313.904022,250.249542,18.996023,4.820197
4,4,0,0.156154,0.248788,0.13725,0.115549,0.089293,10.930715,9.521504,465.421692,35.332596,583.825867,1183.754761,19337768.0,583.825867,465.421692,35.332596,9.521504


In [5]:
### ENSEMBLE SETUP ###
model_2_path = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/SEIRHD_NPI_Type2_petrinet.json"

In [6]:
result_2 = pyciemss.sample(model_2_path, end_time, logging_step_size, num_samples, start_time=start_time)
result_2['data'].head()

,timepoint_id,sample_id,persistent_beta_c_param,persistent_beta_nc_param,persistent_kappa_param,persistent_gamma_param,persistent_hosp_param,persistent_death_hosp_param,persistent_I0_param,D_state_state,E_state_state,H_state_state,I_state_state,R_state_state,S_state_state,infected_observable_state,exposed_observable_state,hospitalized_observable_state,dead_observable_state
0,0,0,0.777164,0.280381,0.712967,0.165812,0.067952,0.068566,3.470171,0.148899,182.210876,3.139023,119.149956,74.865257,19339664.0,119.149956,182.210876,3.139023,0.148899
1,1,0,0.777164,0.280381,0.712967,0.165812,0.067952,0.068566,3.470171,1.718176,1585.308594,28.053324,1037.077759,751.842834,19336630.0,1037.077759,1585.308594,28.053324,1.718176
2,2,0,0.777164,0.280381,0.712967,0.165812,0.067952,0.068566,3.470171,15.415092,13775.308594,244.098328,9016.912109,6641.261230,19310354.0,9016.912109,13775.308594,244.098328,15.415092
3,3,0,0.777164,0.280381,0.712967,0.165812,0.067952,0.068566,3.470171,134.235809,118195.703125,2111.843994,77767.648438,57643.554688,19084188.0,77767.648438,118195.703125,2111.843994,134.235809
4,4,0,0.777164,0.280381,0.712967,0.165812,0.067952,0.068566,3.470171,1139.344849,912600.312500,17470.417969,626840.625000,483028.375000,17298964.0,626840.625000,912600.312500,17470.417969,1139.344849


In [7]:
### ENSEMBLE SAMPLE INTERFACE ###

model_paths = [model_1_path, model_2_path]
solution_mappings = [lambda x : x, lambda x : x] # Conveniently, these two models operate on exactly the same state space, with the same names.

ensemble_result = pyciemss.ensemble_sample(model_paths, solution_mappings, end_time, logging_step_size, num_samples, start_time=start_time)
ensemble_result['data'].head()

,timepoint_id,sample_id,model_0/persistent_beta_c_param,model_0/persistent_kappa_param,model_0/persistent_gamma_param,model_0/persistent_hosp_param,model_0/persistent_death_hosp_param,model_0/persistent_I0_param,model_1/persistent_beta_c_param,model_1/persistent_beta_nc_param,...,model_0/H_state_state,model_0/I_state_state,model_0/R_state_state,model_0/S_state_state,model_1/D_state_state,model_1/E_state_state,model_1/H_state_state,model_1/I_state_state,model_1/R_state_state,model_1/S_state_state
0,0,0,0.228905,0.697764,0.292566,0.098878,0.078968,14.676838,0.746269,0.611267,...,6.609246,82.220551,117.203323,19339694.0,0.022061,30.251297,1.026544,25.661184,50.608181,19339932.0
1,1,0,0.228905,0.697764,0.292566,0.098878,0.078968,14.676838,0.746269,0.611267,...,29.761646,357.304077,639.404541,19338388.0,0.076176,40.174564,1.590585,34.254513,128.773224,19339832.0
2,2,0,0.228905,0.697764,0.292566,0.098878,0.078968,14.676838,0.746269,0.611267,...,129.447113,1552.223877,2909.138916,19332712.0,0.152206,53.485672,2.148299,45.605007,233.071625,19339706.0
3,3,0,0.228905,0.697764,0.292566,0.098878,0.078968,14.676838,0.746269,0.611267,...,561.963379,6735.284180,12763.881836,19308072.0,0.253972,71.206985,2.864267,60.715481,371.954346,19339534.0
4,4,0,0.228905,0.697764,0.292566,0.098878,0.078968,14.676838,0.746269,0.611267,...,2430.759766,29075.619141,55418.121094,19201888.0,0.389529,94.798378,3.813826,80.831390,556.855896,19339306.0


### Sample interface with intervention

In [ ]:
model_3_path = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/SIR_stockflow.json"
start_time = 0.0
end_time = 40.
logging_step_size = 1.0
num_samples = 3
result = pyciemss.sample(model_3_path, end_time, logging_step_size, num_samples=int(5), start_time=start_time, static_parameter_interventions={torch.tensor(1.): {"p_cbeta": torch.tensor(0.5)}}, solver_method="euler")
result["unprocessed_result"]

{'persistent_p_cbeta': tensor([0.4041, 0.4404, 0.3896, 0.4059, 0.3602]),
 'persistent_p_tr': tensor([ 9.8463, 12.9569, 14.2960, 11.1356,  9.6565]),
 'I_state': tensor([[  1.3021,   1.8201,   2.5434,   3.5530,   4.9611,   6.9230,   9.6521,
           13.4405,  18.6838,  25.9109,  35.8155,  49.2819,  67.3906,  91.3753,
          122.4892, 161.7260, 209.3571, 264.3176, 323.6429, 382.3535, 434.2239,
          473.4583, 496.5296, 503.0772, 495.3939, 477.0862, 451.8281, 422.6724,
          391.8654, 360.9314, 330.8413, 302.1746, 275.2471, 250.2020, 227.0722,
          205.8219, 186.3740, 168.6283, 152.4726],
         [  1.3628,   1.9381,   2.7555,   3.9163,   5.5634,   7.8980,  11.2012,
           15.8641,  22.4242,  31.6100,  44.3862,  61.9889,  85.9214, 117.8613,
          159.4036, 211.5562, 273.9606, 343.9989, 416.2750, 483.1879, 536.9528,
          572.2780, 587.9832, 586.5347, 572.2190, 549.4057, 521.6429, 491.4684,
          460.5635, 429.9856, 400.3708, 372.0816, 345.3059, 320.1222, 

### Optimize interface
Get infections below 300 individuals at 100 days for SIR model

In [2]:
from pyciemss.ouu.qoi import scenario2dec_nday_average
import numpy as np

model_3_path = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/SIR_stockflow.json"
start_time = 0.0
end_time = 40.
logging_step_size = 1.0

risk_bound = 300.
qoi = lambda x: scenario2dec_nday_average(x, ["I_state"], 1)
objfun = lambda x: np.abs(x)
static_parameter_interventions = {torch.tensor(1.): "p_cbeta"}
initial_guess_interventions = 0.15
bounds_interventions = [[0.1], [0.5]]

opt_result = pyciemss.optimize(model_3_path, end_time, logging_step_size, qoi, risk_bound, static_parameter_interventions, objfun,
    initial_guess_interventions=initial_guess_interventions, bounds_interventions=bounds_interventions,
    start_time=0.0, n_samples_ouu=int(1e2), maxiter=3, maxfeval=20, solver_method="euler")
opt_result

 35%|███▌      | 28/80 [03:15<06:02,  6.97s/it]


{'policy': 0.1,
 'OptResults':                     message: ['requested number of basinhopping iterations completed successfully']
                     success: True
                         fun: 0.09999999999999998
                           x: 0.1
                         nit: 3
       minimization_failures: 0
                        nfev: 28
  lowest_optimization_result: message: Optimization terminated successfully.
                              success: True
                               status: 1
                                  fun: 0.09999999999999998
                                    x: [ 1.000e-01]
                                 nfev: 7
                                maxcv: 2.7755575615628914e-17}